In [1]:
import os, sys, shutil
import csv, json
import ogr, osr
import logging
from tqdm import tqdm
from collections import defaultdict
from operator import itemgetter
import time
import argparse

import overpy

import xml.etree.ElementTree as ET
import xml.dom.minidom as mn

ogr.UseExceptions()
osr.UseExceptions()

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Set up logging for stdout
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter("[{asctime}][{levelname}][{name}]: {message}", style='{')
handler.setFormatter(formatter)
logging.getLogger().addHandler(handler)

In [5]:
api = overpy.Overpass()

relation_to_area_factor = 3600000000

region_ids = {
	"Laval": 3532125 + relation_to_area_factor,
	"Montreal": 1571328 + relation_to_area_factor
}

service_route_tmpl = """
    area({})->.searchArea;
    (
    relation["type"="route"]["route"="bus"](area.searchArea);
    );
    out body;
"""

In [6]:
routes_result_laval = api.query(service_route_tmpl.format(region_ids['Laval']))

In [7]:
type(routes_result_laval)

overpy.Result

In [9]:
existing_routes = []
values = ['STL', 'Société de transport de Laval']

for relation in routes_result_laval.relations:
    tags = relation.tags
    true_count = 0

    for v in tags.values():
        if v in values:
            true_count += 1

    if true_count > 0:

        relation_member_nodes = []
        relation_member_ways = []

#         if relation.members:
#             for member in relation.members:
#                 if member._type_value == 'node':
#                     relation_member_nodes.append({
#                         "props": {
#                             "type": 'node',
#                             "ref": member.ref,
#                             "role": member.role
#                         }
#                     })
#                 if member._type_value == 'way':
#                     relation_member_ways.append({
#                         "props": {
#                             "type": 'way',
#                             "ref": member.ref,
#                             "role": ""
#                         }
#                     })

        existing_route = {
            "props": {
                "id": relation.id
            },
            "tags": tags,
#             "members": {
#                 "nodes": relation_member_nodes,
#                 "ways": relation_member_ways
#             }
        }

        existing_routes.append(existing_route)

In [10]:
existing_routes

[{'props': {'id': 3886754},
  'tags': {'name': '24 Direction Sainte-Dorothée',
   'operator': 'STL',
   'ref': '24',
   'route': 'bus',
   'type': 'route'}},
 {'props': {'id': 3888221},
  'tags': {'name': 'Direction Cartier',
   'network': 'STL',
   'operator': 'Société de transport de Laval',
   'public_transport:version': '2',
   'ref': '24E',
   'route': 'bus',
   'type': 'route'}},
 {'props': {'id': 3888406},
  'tags': {'from': 'Juge-Desnoyers / Face Au 520',
   'name': 'Direction Métro Cartier',
   'network': 'STL',
   'operator': 'STL',
   'public_transport:version': '2',
   'ref': '12O',
   'roundtrip': 'no',
   'route': 'bus',
   'to': 'Métro Cartier Quai:2',
   'type': 'route'}},
 {'props': {'id': 3888408},
  'tags': {'from': 'Métro Cartier Quai:8',
   'name': 'Direction Chomedey',
   'network': 'STL',
   'operator': 'STL',
   'public_transport:version': '2',
   'ref': '20O',
   'roundtrip': 'no',
   'route': 'bus',
   'to': 'Notre-Dame / Montgolfier',
   'type': 'route'}},
 {

In [16]:
# routes+sorted = sorted(existing_routes, key=itemgetter('props'))

sorted_routes = sorted(existing_routes, key=lambda x: (x['tags']['ref']))

In [33]:
for route in sorted_routes:
    ref = route['tags']['ref']
    
    if ref == '24':
        continue
    
    new_ref = ref[:-1] + ' ' + ref[-1:]
    # print(new_ref)
    new_name = 'Bus: {} - {}'.format(new_ref, route['tags']['name'])
    print(new_name)
    
    route['tags']['name'] = new_name
    route['props']['action'] = 'modify'
        

Bus: 12 E - Bus: 12 E - Direction Pont-Viau
Bus: 12 O - Bus: 12 O - Direction Métro Cartier
Bus: 144 E - Bus: 144 E - Direction Métro Côte-Vertu
Bus: 144 O - Bus: 144 O - Direction Sainte-Dorothée
Bus: 151 N - Bus: 151 N - Direction Sainte-Rose
Bus: 151 S - Bus: 151 S - Direction Métro Côte-Vertu
Bus: 17 N - Bus: 17 N - Direction Auteuil
Bus: 17 S - Bus: 17 S - Direction Métro Cartier
Bus: 20 E - Bus: 20 E - Direction Métro Cartier
Bus: 20 O - Bus: 20 O - Direction Chomedey
Bus: 222 E - Bus: 222 E - Direction Saint-Vincent-De-Paul
Bus: 222 O - Bus: 222 O - Direction Métro Cartier
Bus: 22 E - Bus: 22 E - Direction Saint-François
Bus: 22 O - Bus: 22 O - Direction Métro Cartier
Bus: 24 E - Bus: 24 E - Direction Cartier
Bus: 24 E - Bus: 24 E - Direction Métro Cartier
Bus: 24 O - Bus: 24 O - Direction Sainte-Dorothée
Bus: 252 E - Bus: 252 E - Direction Saint-François
Bus: 252 O - Bus: 252 O - Direction Terminus Henri-Bourassa
Bus: 26 E - Bus: 26 E - Direction Métro Montmorency
Bus: 26 O - B

In [34]:
sorted_routes

[{'props': {'id': 9312810, 'action': 'modify'},
  'tags': {'from': 'Métro Cartier Quai:2',
   'name': 'Bus: 12 E - Bus: 12 E - Direction Pont-Viau',
   'network': 'STL',
   'operator': 'STL',
   'public_transport:version': '2',
   'ref': '12E',
   'roundtrip': 'no',
   'route': 'bus',
   'to': 'Juge-Desnoyers / Face Au 520',
   'type': 'route'}},
 {'props': {'id': 3888406, 'action': 'modify'},
  'tags': {'from': 'Juge-Desnoyers / Face Au 520',
   'name': 'Bus: 12 O - Bus: 12 O - Direction Métro Cartier',
   'network': 'STL',
   'operator': 'STL',
   'public_transport:version': '2',
   'ref': '12O',
   'roundtrip': 'no',
   'route': 'bus',
   'to': 'Métro Cartier Quai:2',
   'type': 'route'}},
 {'props': {'id': 9312809, 'action': 'modify'},
  'tags': {'from': 'Carole / Des Tulipes',
   'name': 'Bus: 144 E - Bus: 144 E - Direction Métro Côte-Vertu',
   'network': 'STL',
   'operator': 'STL',
   'public_transport:version': '2',
   'ref': '144E',
   'roundtrip': 'no',
   'route': 'bus',
  

In [35]:
# logger.info('Writing JOSM XML files for {}.'.format(', '.join(types)))
output_dir = '/home/arthur/PycharmProjects/osm-gtfs-laval/rename'
output_file = os.path.join(output_dir, 'gtfs_laval.xml')

types = ['routes']

if os.path.exists(output_file):
    os.remove(output_file)

root = ET.Element("osm")
root.set('version', '0.6')

# min_lon, max_lon, min_lat, max_lat = self.extent

# bounds = ET.SubElement(root, 'bounds')
# bounds.set('minlat', str(min_lat))
# bounds.set('minlon', str(min_lon))
# bounds.set('maxlat', str(max_lat))
# bounds.set('maxlon', str(max_lon))

# if 'stops' in types:
#     for stop in self.final_stops:
#         node = ET.SubElement(root, 'node')
#         node.set('version', '1')

#         for k, v in stop['tags'].items():
#             tag = ET.SubElement(node, 'tag')
#             tag.set('k', str(k))
#             tag.set('v', str(v))

#         for k, v in stop['props'].items():
#             node.set(k, str(v))

if 'routes' in types:
    for route_relation in sorted_routes:
        relation = ET.SubElement(root, 'relation')
        relation.set('version', '1')

        for k, v in route_relation['tags'].items():
            tag = ET.SubElement(relation, 'tag')
            tag.set('k', str(k))
            tag.set('v', str(v))

        for k, v in route_relation['props'].items():
            relation.set(k, str(v))

#         member_types = route_relation['members'].keys()

#         for member_node in route_relation['members']['nodes']:
#             mem = ET.SubElement(relation, 'member')

#             for k, v in member_node['props'].items():
#                 mem.set(k, str(v))

#         if 'ways' in member_types:
#             for member_way in route_relation['members']['ways']:
#                 mem = ET.SubElement(relation, 'member')

#                 for k, v in member_way['props'].items():
#                     mem.set(k, str(v))

# if 'route_masters' in types:
#     for route_master_relation in self.route_master_relations:
#         relation = ET.SubElement(root, 'relation')
#         relation.set('version', '1')

#         for key, value in route_master_relation.items():
#             if key == 'tags':
#                 for k, v in route_master_relation['tags'].items():
#                     tag = ET.SubElement(relation, 'tag')
#                     tag.set('k', str(k))
#                     tag.set('v', str(v))
#             if key == 'props':
#                 for k, v in route_master_relation['props'].items():
#                     relation.set(k, str(v))
#             if key == 'members':
#                 for member in route_master_relation['members']:
#                     mem = ET.SubElement(relation, 'member')

#                     for key, value in member.items():
#                         if key == 'props':
#                             for k, v in member['props'].items():
#                                 mem.set(k, str(v))

tree = ET.ElementTree(root)
tree.write(output_file, encoding='unicode')